In [2]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from datetime import datetime
import pandas as pd
import re

# Fonction pour extraire et convertir le prix
def extract_and_convert_price(raw_price):
    try:
        # Extraire la partie numérique du prix
        numeric_part = re.search(r'([\d,]+)', raw_price)
        
        if numeric_part:
            numeric_price_str = numeric_part.group(1)
            # Convertir la partie numérique en float en conservant la virgule comme séparateur décimal
            return float(numeric_price_str.replace(',', '.'))
        else:
            return None

    except Exception as e:
        print(f"Une erreur s'est produite lors de l'extraction et de la conversion du prix : {e}")
        return None

# Fonction pour extraire les informations d'un produit à partir de son lien
def scrape_product_info(product_link):
    try:
        response = requests.get(product_link)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        product_name_element = soup.find('h1', {'itemprop': 'name'})
        product_name = product_name_element.text.strip() if product_name_element else None

        image_element = soup.find('img', {'class': 'js-qv-product-cover'})
        image_link = image_element.get('src') if image_element else None

        product_price_element = soup.find('span', {'itemprop': 'price'})
        product_price_raw = product_price_element.text.strip() if product_price_element else None

        # Extraction de la partie numérique du prix
        product_price = extract_and_convert_price(product_price_raw) if product_price_raw else None

        return {
            'name': product_name,
            'nom_site': 'Pharma Shop',
            'image_link': image_link,
            'price': product_price,
            'product_link': product_link
        }

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête pour {product_link}: {e}")
        return None
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'extraction des informations du produit : {e}")
        return None

# Fonction pour extraire les informations de tous les produits d'une page
def scrape_all_products(url, db_config):
    page_number = 1
    products_data = []

    while True:
        current_url = f"{url}?page={page_number}"
        response = requests.get(current_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            product_links = soup.select('.thumbnail.product-thumbnail')

            if not product_links:
                break

            for link in product_links:
                product_link = link.get('href')
                product_info = scrape_product_info(product_link)

                if product_info:
                    products_data.append(product_info)
                    print("Lien du produit:", product_info['product_link'])
                    print("Nom du produit:", product_info['name'])
                    print("Nom du site:", product_info['nom_site'])
                    print("Lien de l'image:", product_info['image_link'])
                    print("Prix du produit:", product_info['price'])
                    print("-" * 50)

            page_number += 1
        else:
            print(f"La requête a échoué avec le code de statut : {response.status_code}")
            break

    df = pd.DataFrame(products_data)
    df.drop_duplicates(subset=['product_link'], keep='first', inplace=True)
    insert_into_database(df, db_config)

# Fonction pour insérer les données dans la base de données MySQL
def insert_into_database(df, db_config):
    try:
        current_date = datetime.today().strftime('%Y-%m-%d')

        with mysql.connector.connect(**db_config) as connection:
            with connection.cursor() as cursor:
                df.drop_duplicates(subset=['product_link'], keep='first', inplace=True)

                for index, row in df.iterrows():
                    cursor.execute("SELECT id_produit FROM produits WHERE lien = %s", (row['product_link'],))
                    existing_product_id = cursor.fetchone()

                    if existing_product_id:
                        cursor.execute("""
                            INSERT INTO prix (id_produit, prix, date_prix)
                            VALUES (%s, %s, %s)
                        """, (existing_product_id[0], row['price'], current_date))
                    else:
                        cursor.execute("""
                            INSERT INTO produits (nom, nom_site, image, lien)
                            VALUES (%s, %s, %s, %s)
                        """, (row['name'], row['nom_site'], row['image_link'], row['product_link']))

                        product_id = cursor.lastrowid

                        cursor.execute("""
                            INSERT INTO prix (id_produit, prix, date_prix)
                            VALUES (%s, %s, %s)
                        """, (product_id, row['price'], current_date))

                connection.commit()
                print("Données insérées dans la base de données MySQL.")
    except mysql.connector.Error as e:
        print(f"Une erreur s'est produite lors de l'insertion dans la base de données : {e}")

# Fonction pour exécuter le scraping et l'insertion de données
def job():
    # URL de la page contenant la liste des produits
    base_url = "https://pharma-shop.tn/839-visage"

    # Configuration de la base de données
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'database': 'scrapping_db',
        'port': 3306
    }

    # Appeler la fonction pour extraire les informations de tous les produits de la première page
    scrape_all_products(base_url, db_config)

# Exécution manuelle
if __name__ == "__main__":
    job()

    # Scheduled Execution
# Uncomment the following lines if you want to schedule the script
# schedule.every().day.at("00:00").do(job)
# while True:
#     schedule.run_pending()
#     time.sleep(1)

Lien du produit: https://pharma-shop.tn/eau-tonique-lotion/4411-dermedic-hydrain-3-eau-micellaire-500-ml-5901643174569.html
Nom du produit: DERMEDIC HYDRAIN 3 EAU MICELLAIRE 500 ML
Nom du site: Pharma Shop
Lien de l'image: https://pharma-shop.tn/6482-large_default/dermedic-hydrain-3-eau-micellaire-500-ml.webp
Prix du produit: 20.5
--------------------------------------------------
Lien du produit: https://pharma-shop.tn/lait-mousse-emulsion/1483-pharmaceris-w-purialbucin-mousse-5900717147010.html
Nom du produit: PHARMACERIS W PURI-ALBUCIN MOUSSE NETTOYANTE ECLAIRCISSANTE, 150ML
Nom du site: Pharma Shop
Lien de l'image: https://pharma-shop.tn/3764-large_default/pharmaceris-w-purialbucin-mousse.webp
Prix du produit: 26.835
--------------------------------------------------
Lien du produit: https://pharma-shop.tn/protection-superieure-a-spf-50/368-filorga-uv-defence-soin-solaire-anti-age-anti-taches-spf50-40ml-3401360202337.html
Nom du produit: FILORGA UV DEFENCE SOIN SOLAIRE ANTI-AGE ANT

In [ ]:
# Scheduled Execution
# Commentez les lignes suivantes si vous ne voulez pas planifier l'exécution du script
# schedule.every().day.at("00:00").do(job)
# while True:
#     schedule.run_pending()
#     time.sleep(1)